In [ ]:
import os
import re
import time
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, WebDriverException

path = "C:\\Users\\lgael\\OneDrive\\Documentos\\Acciona"
print("Directorio de G.G. encontrado.")
Warning = True
url = 'https://xtraders.meteologica.com/'
username = "cacastillot"
password = "Acciona2024*"

def setup_driver(path):

    prefs = {'download.default_directory': path}
    options = webdriver.ChromeOptions()
    options.add_experimental_option('prefs', prefs)
    #options.add_argument('--headless') 
    options.add_argument('--start-maximized')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('prefs', {'credentials_enable_service': False,'profile.password_manager_enabled': False})
    try:
        driver = webdriver.Chrome(options=options)
        return driver
    except WebDriverException as e:
        print(f"Error al configurar el driver: {e}")
        return None

    
def login_to_site(driver):
    try:
        driver.get(url)
        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.ID, "signin")))
        time.sleep(1)

        driver.find_element(By.XPATH, "//*[@id='username']").send_keys(username)
        time.sleep(1)

        driver.find_element(By.XPATH, "//*[@id='password']").send_keys(password)
        time.sleep(1)

        driver.find_element(By.XPATH, "//input[@type='submit' and @value='Login']").click()

        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH, "//h1[@class='title' and text()='Production forecast']")))

        print("Inicio de sesión exitoso.")
        time.sleep(1)

    except TimeoutException:
        print("Error: No es posible iniciar sesión")
        driver.quit()


def wait_for_download(path, timeout=10):

    seconds = 0
    while not any([filename.endswith(".xls") or filename.endswith(".xlsx") for filename in os.listdir(path)]):
        time.sleep(1)
        seconds += 1
        # Si se supera el tiempo de espera
        if seconds > timeout:
            print("Descarga del archivo tomó demasiado tiempo.")
            break
    print("Descarga completada.")


def scroll_and_click(driver, xpath):
    element = driver.find_element(By.XPATH, xpath)
    driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", element)
    time.sleep(1)
    element.click()
    time.sleep(1)

def rename_file_with_asset_and_date(original_file_path, asset_name, path):
    today_date = datetime.now().strftime('%Y-%m-%d')
    file_extension = os.path.splitext(original_file_path)[1]
    new_filename = f"{asset_name}_{today_date}{file_extension}"
    new_file_path = os.path.join(path, new_filename)
    os.rename(original_file_path, new_file_path)
    
    
def select_asset_and_download_csv(driver,asset,path):
    global Warning  
    try:
        scroll_and_click(driver, "//div[@class='facilities header-button txt' and normalize-space()='Assets']")

        #driver.find_element(By.XPATH, "//div[@class='facilities header-button txt' and normalize-space()='Assets']").click()
        time.sleep(3)
        scroll_and_click(driver, f"//div[@id='content' and text()='{asset}']")

        #driver.find_element(By.XPATH, f"//div[@id='content' and text()='{asset}']").click()
        time.sleep(5)

        elements = driver.find_elements(By.XPATH, "//iron-icon[@class='header-button edit button_desplegable x-scope iron-icon-0' and @icon='xtraders:dots']")

        for i in [0,1,2,len(elements)-1]:
            driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", elements[i])
            time.sleep(2)
            elements[i].click()
            time.sleep(3)
            driver.find_element(By.XPATH, "//li[@class='exportar_csv' and contains(text(), 'Export to CSV')]").click()
            time.sleep(2)


            if Warning:
                try:
                    driver.find_element(By.XPATH, "//span[@class='xt-btn flat right aceptar-alternativo activo' and text()=\"Don't show this message again\"]").click()
                    Warning = False  
                    print("Clic en 'Don't show this message again' ejecutado.")
                except NoSuchElementException:
                    print("El botón Warning no se encontro'")

            time.sleep(3)

            scroll_and_click(driver, "//span[@class='xt-btn flat right boton aceptar boton_activo' and text()='Confirm']")

            #driver.find_element(By.XPATH, "//span[@class='xt-btn flat right boton aceptar boton_activo' and text()='Confirm']").click()
            
            original_file_path = wait_for_download(path)
            rename_file_with_asset_and_date(original_file_path, asset, path)

            filename = max([path + "\\" + f for f in os.listdir(path)], key=os.path.getctime)
            print(f'Archivo descargado: {filename}')
            time.sleep(2)

        driver.execute_script("arguments[0].scrollIntoView({block: 'center', inline: 'center'});", elements[0])
        time.sleep(3)

    
    except (TimeoutException, NoSuchElementException, ElementClickInterceptedException) as e:
  
        print(f"Error: No es posible seleccionar el activo o exportar los datos. Error: {e}")


def main():
    
    driver = setup_driver(path)
    if driver is None:
        return

    try:
        login_to_site(driver)
        select_asset_and_download_csv(driver, "C02185 - Calabaza Fase I",path)
        select_asset_and_download_csv(driver, "CHIRIP - Chiripa",path)
        select_asset_and_download_csv(driver, 'CORTAM - El Cortijo Tamaulipas',path)
        select_asset_and_download_csv(driver, 'CORTII - Santa Cruz',path)
        select_asset_and_download_csv(driver, 'OAXII - Oaxaca 2',path)
        select_asset_and_download_csv(driver, 'OAXIII - Oaxaca 3',path)
        select_asset_and_download_csv(driver, 'OAXIV - Oaxaca 4',path)
        select_asset_and_download_csv(driver, 'PL2FII - Tuto Energy II (FV Puerto Libertad)',path)
        select_asset_and_download_csv(driver, 'PUELIB - AT SOLAR V (FV Puerto Libertad)',path)
        select_asset_and_download_csv(driver, 'SANANT - Eurus',path)
        select_asset_and_download_csv(driver, 'SANLAZ - San Carlos',path)
        print("Programa finalizado con exito")
    except Exception as e:
        print(f"Se produjo un error en la ejecución {e}")
    finally:
       driver.quit()

 
        



if __name__ == "__main__":
    main()
